In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from threading import Thread
from time import sleep, time
from urllib.request import urlopen, Request

# Informações Importantes

[Limite de requisições](https://developer.themoviedb.org/docs/rate-limiting)

In [2]:
tmdb = "https://www.themoviedb.org"

In [3]:
h = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

# Classe Item

In [4]:
class Item:
    
    def __init__(self, url=None, tipo='filme', id=None, name=None, html=None, download=True):
        self.tipo = tipo
        if url != None and download:
            self.url = url
            aux = url.replace(tmdb, '')
            self.id = int(aux.split('/')[-1]) if tipo == 'filme' else int(aux.split('/')[2].split('-')[0])

            code = 0
            while code != 200:
                r = requests.get(url, headers=h)
                code = r.status_code
                if code != 200:
                    sleep(2)
            self.html = bs(r.text, 'html.parser')
            self.name = self.html.find('div', {'class': 'title'}).find('a').text if tipo == 'filme' else self.html.find('h2', {'class': 'title'}).text
        else:
            self.id = id
            self.name = name
            self.html = html
            self.url = url
    
    def pegaId(self):
        return self.link.split('/')[-2] if self.tipo == 'filme' else self.link.split('/')[2].split('-')[0]
    
    def __str__(self):
        return f'{self.id} - {self.name} - {self.url} - {self.tipo} - html = {"TEM" if self.html != None else "Nao tem"}'

## Classe Filme

O filme deve possuir uma lista com seus principais atores, que virá no HTML.

In [5]:
class Filme(Item):
    def __init__(self, url):
        super().__init__(url=url, tipo='filme')
        aux = [(ator.find('a')['href'].split('/')[-1].split('-')[0], ator.find('p').text) 
               for ator in self.html.find_all('ol', {'class': 'people scroller'})[0].find_all('li')][:-1]
        self.elenco = [
            Item(
                id=int(x[0]),
                name=x[1], 
                tipo='ator', 
                url=f'{tmdb}/person/{x[0]}', 
                download=False) for x in aux
            ]

## Classe Ator

Um ator deve possuir uma lista com seus principais filmes, que virá no HTML.

In [6]:
class Ator(Item):
    def __init__(self, url):
        super().__init__(url=url, tipo='ator')
        ids = [ (
                    int(filme.find('a')['href'].split('/')[-1]),
                    filme.find('p').text
                ) for filme in self.html.find_all('div', id='known_for')[0].find_all('li')]
        self.know = [ 
            Item(
                id=i[0],
                name=i[1], 
                tipo='filme', 
                url=f"{tmdb}/movie/{i[0]}", 
                download=False) for i in ids
            ]

# Dois Atores que quero conectar

In [7]:
urlS1 = "https://www.themoviedb.org/person/10055-fernanda-montenegro"
urlS2 = "https://www.themoviedb.org/person/5064-meryl-streep"

In [8]:
s1, s2 = Ator(url=urlS1), Ator(url=urlS2)

In [9]:
print(s1); print(s2)

10055 - Fernanda Montenegro - https://www.themoviedb.org/person/10055-fernanda-montenegro - ator - html = TEM
5064 - Meryl Streep - https://www.themoviedb.org/person/5064-meryl-streep - ator - html = TEM


In [10]:
for i in s1.know:
    print(i)

666 - Central Station - https://www.themoviedb.org/movie/666 - filme - html = Nao tem
548544 - Invisible Life - https://www.themoviedb.org/movie/548544 - filme - html = Nao tem
40096 - A Dog's Will - https://www.themoviedb.org/movie/40096 - filme - html = Nao tem
223381 - Time and the Wind - https://www.themoviedb.org/movie/223381 - filme - html = Nao tem
6639 - Love in the Time of Cholera - https://www.themoviedb.org/movie/6639 - filme - html = Nao tem
29262 - The House of Sand - https://www.themoviedb.org/movie/29262 - filme - html = Nao tem
765613 - Emicida: AmarElo - It's All for Yesterday - https://www.themoviedb.org/movie/765613 - filme - html = Nao tem
258521 - The Deceased - https://www.themoviedb.org/movie/258521 - filme - html = Nao tem


In [11]:
filme_teste = "https://www.themoviedb.org/movie/843847"

In [12]:
f  = Filme(url=filme_teste)

In [13]:
for x in f.elenco:
    print(x)

17419 - Bryan Cranston - https://www.themoviedb.org/person/17419 - ator - html = Nao tem
516 - Annette Bening - https://www.themoviedb.org/person/516 - ator - html = Nao tem
11678 - Rainn Wilson - https://www.themoviedb.org/person/11678 - ator - html = Nao tem
154091 - Larry Wilmore - https://www.themoviedb.org/person/154091 - ator - html = Nao tem
21731 - Michael McKean - https://www.themoviedb.org/person/21731 - ator - html = Nao tem
196181 - Ann Harada - https://www.themoviedb.org/person/196181 - ator - html = Nao tem
94791 - Jake McDorman - https://www.themoviedb.org/person/94791 - ator - html = Nao tem
221098 - Anna Camp - https://www.themoviedb.org/person/221098 - ator - html = Nao tem
2551620 - Devyn McDowell - https://www.themoviedb.org/person/2551620 - ator - html = Nao tem


# Inicio

In [14]:
mensagem = lambda filme, ator: f"O ator {ator.name} participou do filme {filme.name}"

In [15]:
def busca(item, lista):
    r = [x for x in lista if x.id == item.id]
    return len(r) > 0, r[0] if len(r) > 0 else None

In [16]:
def escolheAtor(atual:Filme, historico:list):
    escolhido = None
    i = 0
    while escolhido == None and i < len(atual.elenco):
        if busca(item=atual.elenco[i], lista=historico)[0] == False:
            escolhido = atual.elenco[i]
        i += 1
    return escolhido

In [17]:
def escolheFilme(atorAtual:Ator, historico:list):
    escolhido = None
    i = 0
    while escolhido == None and i < len(atorAtual.know):
        if busca(item=atorAtual.know[i], lista=historico)[0] == False:
            escolhido = atorAtual.know[i]
        i += 1
    return escolhido

In [18]:
def buscaSeq(movInit:Item, s2:Ator, seq:dict, thr:int, numMax:int=100, possivel:bool=True):
    # Possivel = True: Ainda é possivel encontrar o caminho.
    # Possivel = False: Não é mais possivel encontrar o caminho pelos atores principais.
    # Acrenscentar uma trava a cada 35 requisições, já que o site só permite 40 por segundo.
    while len(seq['historico']) < numMax and seq['encontrou'] == False and possivel:
        seq['historico'].append(Filme(url=movInit.url)) # Requisitar o filme e já colocar no historico.
        # print(f"T{thr} | FILME: {seq['historico'][-1].name}")
        encontrou, ator = busca(item=s2, lista=seq['historico'][-1].elenco)

        # Encontrou?
        if encontrou: # Sim:
            # print(f"T{thr} | {mensagem(filme=seq['historico'][-1], ator=ator)}")
            seq['historico'].append(s2)
            seq['encontrou'] = True

        else: # Não:

            # Escolher o ator entre o elenco, deve ser alguém que não esteja no historico.
            atorAtual = escolheAtor(atual=seq['historico'][-1], historico=seq['historico'])

            if atorAtual != None: # Ainda é possivel encontrar o caminho.
                seq['historico'].append(Ator(url=atorAtual.url))
                # print(f"T{thr} | ATOR: {seq['historico'][-1].name}")

                # Escolher o filme entre os filmes que o ator participou, deve ser um filme que não esteja no historico.
                movInit = escolheFilme(atorAtual=seq['historico'][-1], historico=seq['historico'])

            else:
                possivel = False
        print(f"T{thr} | {len(seq['historico'])}")

    print(f"T{thr} | FIM DA THREAD | {'ENCONTRADO' if seq['encontrou'] else 'NAO ENCONTRADO'}")

In [19]:
def conecta(s1, s2, qntThrs=1):
    numMax = 100
    encontrou = False
    sequencias = [{'encontrou': False, 'historico': [s1]} for _ in range(qntThrs)]

    encontrados = 0
    thrs = [Thread(target=buscaSeq, args=(s1.know[i], s2, sequencias[i], i, numMax)) for i in range(qntThrs)]
    
    # print(thrs)

    for thr in thrs:
        thr.start()

    for thr in thrs:
        thr.join()

    for seq in sequencias:
        if seq['encontrou']:
            encontrados += 1

    return sequencias, encontrados

In [20]:
seqs, encontrados = conecta(s1, s2, qntThrs=8)

T2 | 3
T7 | 3
T0 | 3
T6 | 3
T3 | 3
T4 | 3T1 | 3

T5 | 3
T2 | 5
T7 | 5
T6 | 5
T4 | 5
T3 | 5
T1 | 5
T5 | 5
T2 | 7
T7 | 7
T4 | 7
T3 | 7
T5 | 7
T6 | 7
T1 | 7
T7 | 9
T4 | 9
T3 | 9
T2 | 9
T5 | 9
T6 | 9
T1 | 9
T4 | 11
T7 | 11
T3 | 11
T2 | 11
T5 | 11
T6 | 11
T2 | 13
T1 | 11
T4 | 13
T5 | 13
T7 | 13
T6 | 13
T4 | 15
T5 | 15
T7 | 15
T1 | 13
T2 | 15
T6 | 15
T4 | 17
T5 | 17
T7 | 17
T1 | 15
T6 | 17
T2 | 17
T5 | 19
T4 | 19
T1 | 17
T7 | 19
T6 | 19
T2 | 19
T1 | 19
T4 | 21
T6 | 21
T2 | 21
T1 | 21
T4 | 23
T6 | 23
T1 | 23
T4 | 25
T2 | 23
T1 | 25
T6 | 25
T4 | 27
T2 | 25
T4 | 29
T1 | 27
T6 | 27
T2 | 27
T4 | 31
T1 | 29
T6 | 29
T2 | 29
T1 | 31
T6 | 31
T4 | 33
T2 | 31
T6 | 33
T6 | FIM DA THREAD | ENCONTRADO
T1 | 33
T4 | 35
T2 | 33
T1 | 35
T4 | 37
T2 | 35
T1 | 37
T4 | 39
T1 | 39
T2 | 37
T4 | 41
T1 | 41
T2 | 39
T4 | 43
T2 | 41
T1 | 43
T4 | 45
T2 | 43
T1 | 45
T4 | 47
T2 | 45
T1 | 47
T4 | 49
T2 | 47
T1 | 49
T2 | 49
T4 | 51
T2 | 51
T2 | FIM DA THREAD | ENCONTRADO
T4 | 53
T1 | 51
T4 | 55
T1 | 53
T4 | 57
T1 | 55
T4 | 

KeyboardInterrupt: 

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/usr/lib/python3/dist-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 665, in urlopen
    httplib_response = self._make_request(
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 376, in _make_request
    self._validate_conn(conn)
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", lin

In [ ]:
encontrados

1